## Create Wav2Vec2CTCTokenizer

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [2]:
all_data = load_dataset('csv',data_files='./order_speech_ko1000.csv',split='train')

Using custom data configuration default-722dfaf1004ac490
Reusing dataset csv (C:\Users\AI_server\.cache\huggingface\datasets\csv\default-722dfaf1004ac490\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


In [3]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [4]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-722dfaf1004ac490\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-cc8bd026da9643a4.arrow


In [5]:
df = pd.DataFrame(remove_spectial_char_data)
df.head()

,src,text
0,script1_g_0044-6001-01-01-KSM-F-05-A.wav,보고 있는 영상 정지시켜 줘
1,script1_g_0044-6002-01-01-KSM-F-05-A.wav,다음 주까지 날씨가 어때
2,script1_g_0044-6003-01-01-KSM-F-05-A.wav,나 대신 점등해 줘
3,script1_g_0044-6004-01-01-KSM-F-05-A.wav,이번 주 대체로 흐린지 궁금해
4,script1_g_0044-6005-01-01-KSM-F-05-A.wav,지금 당장 취침 등 꺼 줘


In [6]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [7]:
char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
char_vocab

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})

In [9]:
vocab_list = list(set(char_vocab["vocab"][0]))

In [10]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'바': 0,
 '캐': 1,
 '라': 2,
 '오': 3,
 '냄': 4,
 '녹': 5,
 '큰': 6,
 '히': 7,
 '먹': 8,
 '급': 9,
 '무': 10,
 '량': 11,
 '없': 12,
 '쓰': 13,
 ' ': 14,
 '중': 15,
 '챙': 16,
 '깜': 17,
 '울': 18,
 '딘': 19,
 '퍼': 20,
 '연': 21,
 '성': 22,
 '균': 23,
 '즌': 24,
 '춤': 25,
 '더': 26,
 '걸': 27,
 '종': 28,
 '린': 29,
 '잠': 30,
 '십': 31,
 '주': 32,
 '빛': 33,
 '흥': 34,
 '떤': 35,
 '밤': 36,
 '찾': 37,
 '방': 38,
 '람': 39,
 '천': 40,
 '슨': 41,
 '아': 42,
 '크': 43,
 '위': 44,
 '봅': 45,
 '핫': 46,
 '내': 47,
 '마': 48,
 '토': 49,
 '몇': 50,
 '생': 51,
 '만': 52,
 '벽': 53,
 '넣': 54,
 '콘': 55,
 '한': 56,
 '원': 57,
 '터': 58,
 '헤': 59,
 '포': 60,
 '엔': 61,
 '률': 62,
 '닫': 63,
 '같': 64,
 '떻': 65,
 '많': 66,
 '하': 67,
 '치': 68,
 '싫': 69,
 '뀌': 70,
 '필': 71,
 '력': 72,
 '광': 73,
 '열': 74,
 '려': 75,
 '도': 76,
 '씨': 77,
 '을': 78,
 '롤': 79,
 '산': 80,
 '듣': 81,
 '태': 82,
 '져': 83,
 '름': 84,
 '총': 85,
 '막': 86,
 '풀': 87,
 '맑': 88,
 '전': 89,
 '났': 90,
 '쉬': 91,
 '퇴': 92,
 '팔': 93,
 '황': 94,
 '떼': 95,
 '플': 96,
 '끝': 97,
 '최': 98,
 '텐': 99,
 '프': 100,

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

531

In [12]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [13]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [14]:
import torch

In [15]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [16]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [17]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [18]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [19]:
import librosa

In [20]:

def voice_sep(sig):
    sig = np.array(sig).flatten()
    S_full, phase = librosa.magphase(librosa.stft(sig))
    S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
    S_filter = np.minimum(S_full, S_filter)
    margin_v = 2
    power = 2
    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)
    S_foreground = mask_v * S_full
    y_foreground = librosa.istft(S_foreground * phase)
    return y_foreground

def load_audio(batch):
    batch['array'],_ = librosa.load('./dataset/audio/'+batch['src'],sr=16000)
    batch['array'] = voice_sep(batch['array'])
    
    return batch

In [21]:
audio_data = remove_spectial_char_data.map(load_audio)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-722dfaf1004ac490\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-baae49471d76c12f.arrow


## Preprocess Data

In [22]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [23]:
audio_data

Dataset({
    features: ['src', 'text', 'array'],
    num_rows: 1000
})

In [24]:
order_voice = audio_data.map(
    prepare_dataset,
    remove_columns=remove_spectial_char_data.column_names,
    # num_proc=4
)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [25]:
order_voice

Dataset({
    features: ['array', 'input_values', 'labels'],
    num_rows: 1000
})

# Training

## Set-up Trainer

In [26]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [27]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [28]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [29]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [30]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [31]:
model.freeze_feature_extractor()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [32]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [33]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=100,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [34]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(order_voice)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(order_voice,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [35]:
torch.cuda.is_available()

True

In [36]:
trainer.train()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: array. If array are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.


{'loss': 51.3664, 'learning_rate': 4.8e-06, 'epoch': 0.4}
{'loss': 52.9128, 'learning_rate': 1.0799999999999998e-05, 'epoch': 0.8}
{'loss': 50.8141, 'learning_rate': 1.68e-05, 'epoch': 1.2}
{'loss': 51.2312, 'learning_rate': 2.28e-05, 'epoch': 1.6}
{'loss': 49.2999, 'learning_rate': 2.8199999999999998e-05, 'epoch': 2.0}
{'loss': 44.1223, 'learning_rate': 3.42e-05, 'epoch': 2.4}
{'loss': 37.6529, 'learning_rate': 3.96e-05, 'epoch': 2.8}
{'loss': 29.5956, 'learning_rate': 4.56e-05, 'epoch': 3.2}
{'loss': 26.2828, 'learning_rate': 5.1599999999999994e-05, 'epoch': 3.6}


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


...

Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-2500
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2500\config.json


{'eval_loss': 0.9267768859863281, 'eval_wer': 0.7187196896217265, 'eval_runtime': 11.61, 'eval_samples_per_second': 17.227, 'eval_steps_per_second': 2.153, 'epoch': 100.0}


Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2500\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2500\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-2300] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 7706.641, 'train_samples_per_second': 10.381, 'train_steps_per_second': 0.324, 'train_loss': 3.5201705753326418, 'epoch': 100.0}


TrainOutput(global_step=2500, training_loss=3.5201705753326418, metrics={'train_runtime': 7706.641, 'train_samples_per_second': 10.381, 'train_steps_per_second': 0.324, 'train_loss': 3.5201705753326418, 'epoch': 100.0})

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.